In [ ]:
%pip install sentencepiece

In [430]:
#!g1.1
import torch

import pandas as pd
import numpy as np
import math

import nltk

from nltk.tokenize import word_tokenize, wordpunct_tokenize


%pip install --upgrade transformers datasets accelerate deepspeed
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
# import datasets

Defaulting to user installation because normal site-packages is not writeable
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
moto 1.3.14 requires idna<2.9,>=2.5, but you have idna 3.4 which is incompatible.
kaggle 1.5.8 requires urllib3<1.25,>=1.21.1, but you have urllib3 1.26.15 which is incompatible.
cloud-ml 0.0.1 requires requests<=2.25.1,>=2.22.0, but you have requests 2.28.2 which is incompatible.
cloud-ml 0.0.1 requires tqdm<=4.54.1,>=4.45.0, but you have tqdm 4.65.0 which is incompatible.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [431]:
#!g1.1
import matplotlib.pyplot as plt
%matplotlib inline
import json

In [432]:
#!g1.1
model_name = "Tatyana/rubert-base-cased-sentiment-new"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

In [433]:
#!g1.1
DATA_PATH = '/home/jupyter/mnt/s3/dataset_hackaton/'

In [ ]:
!ls /home/jupyter/mnt/s3/dataset_hackaton/

In [434]:
#!g1.1
train_df = pd.read_csv(DATA_PATH + 'train_OHE.csv')

bank_categories_df = pd.read_csv(DATA_PATH + 'bank-sentiment.csv')
bank_sentiment_df = pd.read_csv(DATA_PATH + 'bank-categories.csv')


In [435]:
#!g1.1
train_df

,Unnamed: 0,Unnamed: 0.1,sentence,1category,2category,sentiment
0,0,4754,При этом всегда получал качественные услуги.,"[0, 1, 0, 0, 0]","[0, 0, 0, 0, 0]","[1, 0, 0]"
1,1,4417,"Не вижу, за что хотя бы 2 поставить, сервис на 1!","[1, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 1]"
2,2,3629,"Вот так ""Мой любимый"" банк МКБ меня обманул.","[1, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 1]"
3,3,11640,Отвратительное отношение к клиентам.,"[0, 1, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 1]"
4,4,5571,"Всегда в любое время дня и ночи помогут, ответ...","[0, 1, 0, 0, 0]","[0, 0, 0, 0, 0]","[1, 0, 0]"
...,...,...,...,...,...,...
19356,19356,8004,Никогда и ни в коем случае не открывайте счет ...,"[0, 1, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 1]"
19357,19357,18182,ТИ откровенно забили на качество и развивают с...,"[0, 0, 0, 1, 0]","[0, 0, 0, 0, 0]","[0, 0, 1]"
19358,19358,744,"Я считаю, это прорыв и лидерство финансовых ус...","[1, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[1, 0, 0]"
19359,19359,6220,"Писал мужчина очень доходчиво, не финансовым я...","[0, 1, 0, 0, 0]","[0, 0, 0, 0, 0]","[1, 0, 0]"


In [436]:
#!g1.1
# indexes = np.arange(train_df.shape[0])
# np.random.shuffle(indexes)
# train_size = math.ceil(train_df.shape[0] * 0.8)

# train_indexes = indexes[:train_size]
# test_indexes = indexes[train_size:]

# train_data = train_df.iloc[train_indexes]
# test_data = train_df.iloc[test_indexes]

# train_data.reset_index(drop=True, inplace=True)
# test_data.reset_index(drop=True, inplace=True)


indexes = np.arange(train_df.shape[0])
np.random.shuffle(indexes)
train_size = math.ceil(train_df.shape[0] * 0.8)

data = train_df.iloc[indexes]

In [437]:
#!g1.1
MAX_LENGTH = 128
def preprocess_function(examples):
    result = tokenizer(
        examples,
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
#     result['target'] = examples['target']
    return result

In [438]:
#!g1.1
from torch.utils.data import Dataset, random_split


class Dataset(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_columns: list, tokenizer):  
        self.data = data
        self.tokenizer = tokenizer

        self.feature_column = feature_column
        self.target_columns = target_columns
        self.label2list = lambda x: [int(i) for i in json.loads(x)]

    def __getitem__(self, item):
        indx_tokens = preprocess_function(self.data[self.feature_column][item])
        d = {}
        for label in self.target_columns:
            d[f'target_{label}'] = self.label2list(self.data[label][item])
            
        return {**indx_tokens, **d}

    def get_tokens_(self, text):
          return self.tokenizer(text)

    def __len__(self):
        return self.data.shape[0]
    
    

In [439]:
#!g1.1
train_df

,Unnamed: 0,Unnamed: 0.1,sentence,1category,2category,sentiment
0,0,4754,При этом всегда получал качественные услуги.,"[0, 1, 0, 0, 0]","[0, 0, 0, 0, 0]","[1, 0, 0]"
1,1,4417,"Не вижу, за что хотя бы 2 поставить, сервис на 1!","[1, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 1]"
2,2,3629,"Вот так ""Мой любимый"" банк МКБ меня обманул.","[1, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 1]"
3,3,11640,Отвратительное отношение к клиентам.,"[0, 1, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 1]"
4,4,5571,"Всегда в любое время дня и ночи помогут, ответ...","[0, 1, 0, 0, 0]","[0, 0, 0, 0, 0]","[1, 0, 0]"
...,...,...,...,...,...,...
19356,19356,8004,Никогда и ни в коем случае не открывайте счет ...,"[0, 1, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 1]"
19357,19357,18182,ТИ откровенно забили на качество и развивают с...,"[0, 0, 0, 1, 0]","[0, 0, 0, 0, 0]","[0, 0, 1]"
19358,19358,744,"Я считаю, это прорыв и лидерство финансовых ус...","[1, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[1, 0, 0]"
19359,19359,6220,"Писал мужчина очень доходчиво, не финансовым я...","[0, 1, 0, 0, 0]","[0, 0, 0, 0, 0]","[1, 0, 0]"


In [440]:
#!g1.1
dataset = Dataset(data, 'sentence', ['1category', '2category', 'sentiment'], tokenizer)

In [253]:
#!g1.1
# dataset[0]

In [92]:
#!g1.1


0

In [441]:
#!g1.1
from torch.utils.data import DataLoader


batch_size = 8
num_workers = 4

def average_emb(batch):
    features = [b["tokens"] for b in batch]
    targets = [b["target"] for b in batch]

    return {"tokens": features, "targets": torch.LongTensor(targets)}


train_size = math.ceil(len(dataset) * 0.8)

# train, valid = random_split(dataset, [train_size, len(train_data) - train_size])
train, valid = random_split(dataset, [train_size, len(dataset) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=transformers.default_data_collator)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=transformers.default_data_collator)

In [442]:
#!g1.1
list(next(iter(train_loader)).keys())

['input_ids',
 'token_type_ids',
 'attention_mask',
 'target_1category',
 'target_2category',
 'target_sentiment']

In [443]:
#!g1.1
newmodel = torch.nn.Sequential(*(list(model.children())[:-1]))[0]

In [427]:
#!g1.1
# newmodel

In [428]:
#!g1.1


In [429]:
#!g1.1


In [444]:
#!g1.1
def freeze_layers(model, num_to_freeze=0):
    
    for layer in list(model.parameters())[:-num_to_freeze]:
        layer.requires_grad_ = False
    return model
        

In [529]:
#!g1.1


class Cls_head(nn.Module):
    
    def __init__(self, last_layer_dim=1024, hidden_layer_size=512, out_size=1):
        super().__init__()
        self.last_layer_dim = last_layer_dim
        self.hidden_layer_size = hidden_layer_size,
        self.out_size = out_size
        
        self.hidden_layer_1 = nn.Linear(last_layer_dim, hidden_layer_size)
        self.relu1 = nn.ReLU(inplace=True)
        self.bn_1 = nn.BatchNorm1d(hidden_layer_size)
        
        self.hidden_layer_2 = nn.Linear(hidden_layer_size, hidden_layer_size)
        self.relu2 = nn.ReLU(inplace=True)
        self.bn_2 = nn.BatchNorm1d(hidden_layer_size)
        
        
        self.hidden_layer_3 = nn.Linear(hidden_layer_size, hidden_layer_size // 4)
        self.relu3 = nn.ReLU(inplace=True)
        self.bn_3 = nn.BatchNorm1d(hidden_layer_size // 4)
        
        self.cls = nn.Linear(hidden_layer_size // 4, out_size)
        
        
        
    def forward(self, x):
        
        out = self.bn_1(self.relu1(self.hidden_layer_1(x)))
        out = self.bn_2(self.relu2(self.hidden_layer_2(out)))
        out = self.bn_3(self.relu3(self.hidden_layer_3(out)))
        out = self.cls(out)
        
        return out
        

class FBERTAcls(nn.Module):

    def __init__(self, model, last_layer_dim=768, hidden_layer_size=256):
        super().__init__()
        
        
        self.BERT = freeze_layers(model, 0)
        
        self.cls_head_1 = Cls_head(last_layer_dim=last_layer_dim, hidden_layer_size=hidden_layer_size, out_size=3)
        self.cls_head_2 = Cls_head(last_layer_dim=last_layer_dim, hidden_layer_size=hidden_layer_size, out_size=5)
        
        
        

    def forward(self, input_ids, attention_mask, token_type_ids=None):
        bert_output = self.BERT(input_ids=input_ids, attention_mask=attention_mask)
#         hidden_out = self.hidden_layer_1(bert_output.last_hidden_state[:, 0, :])

#         print(bert_output.shape)

        bert_output = bert_output.last_hidden_state[:, 0, :]
#         print(bert_output.shape)
        print(bert_output.sum())
        sent_logits = self.cls_head_1(bert_output) 
        
        cat_1_logits = self.cls_head_2(bert_output)

        return sent_logits, cat_1_logits




In [530]:
#!g1.1
model = FBERTAcls(newmodel, last_layer_dim=768, hidden_layer_size=1024).cpu()

In [531]:
#!g1.1
device = 'cuda'

In [532]:
#!g1.1
batch = next(iter(train_loader))
list(batch.keys())

['input_ids',
 'token_type_ids',
 'attention_mask',
 'target_1category',
 'target_2category',
 'target_sentiment']

In [533]:
#!g1.1
model(batch['input_ids'], batch['attention_mask'])

tensor(-156.4062, grad_fn=<SumBackward0>)


(tensor([[-1.0024,  0.2347, -0.3949],
         [ 0.3470, -0.1218, -0.8285],
         [-0.5675, -0.2159,  0.3164],
         [ 0.6167, -0.5360, -1.0351],
         [ 0.2754, -0.2067,  0.5946],
         [ 0.2642,  0.1652,  1.3120],
         [ 0.0483,  0.7264,  0.4687],
         [-0.1964,  0.4268, -0.5441]], grad_fn=<AddmmBackward>),
 tensor([[-0.3497, -0.3287, -0.4689,  1.1853, -1.5885],
         [ 0.5427, -0.5132, -0.3485,  0.0564,  0.0781],
         [-0.2073,  1.1617, -0.0938, -0.2017,  0.1136],
         [-0.1618,  0.6219,  0.3582, -0.5494,  0.0696],
         [ 0.5467, -0.1068, -0.4584, -0.9157,  1.3640],
         [ 0.1283, -0.4869,  0.1815,  0.7330,  0.5572],
         [-0.5337, -0.3652, -0.2505,  0.1101, -0.7109],
         [-0.3056,  0.4062,  0.7611, -0.7672,  0.4678]],
        grad_fn=<AddmmBackward>))

In [534]:
#!g1.1
def compute_loss(logits_sent, logits_1cat, targets_sent, targets_1cat):
    ce_loss = nn.CrossEntropyLoss()
    
    
    loss = ce_loss(logits_sent, targets_sent) + 0.5 * ce_loss(logits_1cat, targets_1cat)
    
    return loss
    
    

In [535]:
#!g1.1
from tqdm.notebook import tqdm

# ['input_ids',
#  'token_type_ids',
#  'attention_mask',
#  'target_1category',
#  'target_2category',
#  'target_sentiment']


def training(model, optimizer, criterion, train_loader, epoch, device="cpu"):
    pbar = tqdm(train_loader, desc=f"Epoch {epoch + 1}. Train Loss: {0}")
    
    model.train()
    
    total_loss = 0
    steps = 0
    
    count = 0
    for batch in pbar:
        optimizer.zero_grad()
        
        features = batch["input_ids"].to(device)
        attn_mask = batch['attention_mask'].to(device)
        
        targets_sent = torch.argmax(batch["target_sentiment"].to(device), dim=-1)
        targets_1cat = torch.argmax(batch["target_1category"].to(device), dim=-1)

        sent_logits, cat_logits = model(features, attn_mask)
#         print(sent_logits)
        
        loss = compute_loss(sent_logits, cat_logits, targets_sent, targets_1cat)
        
        loss.backward()

        optimizer.step()
        
        total_loss += loss.detach().cpu()
        
        pbar.set_description(f"Epoch {epoch + 1}. Train Loss: {loss.detach().cpu():.4}")
        steps += 1
        
    return total_loss / steps


In [536]:
#!g1.1
def testing(model, criterion, test_loader, threshold=0.5, device="cpu"):
    pbar = tqdm(test_loader, desc=f"Test Loss: {0}, Test Acc: {0}")
    mean_loss = 0
    mean_acc = 0
    model.eval()
    
    
    all_probs_sent = []
    all_probs_1cat = []
    
    target_labels_sent = []
    target_labels_sent_1cat = []
    
    with torch.no_grad():
        for batch in pbar:
            
            features = batch["input_ids"].to(device)
            attn_mask = batch['attention_mask'].to(device)
        
            targets_sent = torch.argmax(batch["target_sentiment"].to(device), dim=-1)
            targets_1cat = torch.argmax(batch["target_1category"].to(device), dim=-1)

            sent_logits, cat_logits = model(features, attn_mask)
            print(sent_logits)
            
            target_labels_sent.append(batch["target_sentiment"].cpu().numpy())
            target_labels_sent_1cat.append(batch["target_sentiment"].cpu().numpy())
            
            all_probs_sent.append(torch.softmax(sent_logits, dim=-1).cpu().numpy())
            all_probs_1cat.append(torch.softmax(cat_logits, dim=-1).cpu().numpy())
            
            loss = compute_loss(sent_logits, cat_logits, targets_sent, targets_1cat)

#             predict = probas.cpu() > threshold
#             acc = (predict.cpu() == targets.cpu()).sum() / len(targets)

            mean_loss += loss.item()
#             mean_acc += acc.item()

#             pbar.set_description(f"Test Loss: {loss:.4}, Test Acc: {acc:.4}")

#     pbar.set_description(f"Test Loss: {mean_loss / len(test_loader):.4}, Test Acc: {mean_acc / len(test_loader):.4}")
    return {'all_probs_sent': all_probs_sent,
            'all_probs_1cat': all_probs_1cat, 
           'target_sent': target_labels_sent,
           'target_1cat': target_labels_sent_1cat}
#     return {"Test Loss": mean_loss / len(test_loader), "Test Acc": mean_acc / len(test_loader), 'target_labels': target_labels, 'probs': all_probs}


In [537]:
#!g1.1

device = 'cuda:0'

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-5)

criterion = nn.BCEWithLogitsLoss()

torch.cuda.is_available()

True

In [538]:
#!g1.1
# batch = next(iter(train_loader))

In [539]:
#!g1.1
losses = []
print(device)

for epoch in range(10):
    loss = training(model, optimizer, criterion, train_loader, epoch, device)
    losses.append(loss)
    plt.plot(list(range(epoch + 1)), losses)
    plt.show()
    
    

cuda:0


Epoch 1. Train Loss: 0:   0%|          | 0/1936 [00:00<?, ?it/s]

tensor(-156.6514, device='cuda:0', grad_fn=<SumBackward0>)
tensor(-156.1871, device='cuda:0', grad_fn=<SumBackward0>)
tensor(-159.8499, device='cuda:0', grad_fn=<SumBackward0>)
tensor(-159.8631, device='cuda:0', grad_fn=<SumBackward0>)
tensor(-163.8573, device='cuda:0', grad_fn=<SumBackward0>)
tensor(-160.0812, device='cuda:0', grad_fn=<SumBackward0>)
tensor(-162.6152, device='cuda:0', grad_fn=<SumBackward0>)
tensor(-158.8139, device='cuda:0', grad_fn=<SumBackward0>)
tensor(-159.7565, device='cuda:0', grad_fn=<SumBackward0>)
tensor(-159.5452, device='cuda:0', grad_fn=<SumBackward0>)
tensor(-154.8857, device='cuda:0', grad_fn=<SumBackward0>)


╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│ <ipython-input-110-6edc8f719c6e>:5 in <module>                                            │
│ <ipython-input-106-a53203919001>:34 in training                                           │
│                                                                                           │
│ /usr/local/lib/python3.8/dist-packages/torch/_tensor.py:255 in backward                   │
│                                                                                           │
│    252 │   │   │   │   retain_graph=retain_graph,                                         │
│    253 │   │   │   │   create_graph=create_graph,                                         │
│    254 │   │   │   │   inputs=inputs)                                                     │
│ ❱  255 │   │   torch.autograd.backward(self, gradient, retain_graph, create_graph, inputs │
│    256 │                                                                                  │
│    257 │   def register_hook(self, hook):                                                 │
│    258 │   │   r"""Registers a backward hook.                                             │
│                                                                                           │
│ /usr/local/lib/python3.8/dist-packages/torch/autograd/__init__.py:147 in backward         │
│                                                                                           │
│   144 │   if retain_graph is None:                                                        │
│   145 │   │   retain_graph = create_graph                                                 │
│   146 │                                                                                   │
│ ❱ 147 │   Variable._execution_engine.run_backward(                                        │
│   148 │   │   tensors, grad_tensors_, retain_graph, create_graph, inputs,                 │
│   149 │   │   allow_unreachable=True, accumulate_grad=True)  # allow_unreachable flag     │
│   150                                                                                     │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt

In [540]:
#!g1.1
t  = testing(model, criterion, train_loader, 0.5, device)

Test Loss: 0, Test Acc: 0:   0%|          | 0/1936 [00:00<?, ?it/s]

tensor(-159.6098, device='cuda:0')
tensor([[ 0.0002, -0.0375,  0.0551],
        [ 0.0002, -0.0375,  0.0551],
        [ 0.0002, -0.0375,  0.0551],
        [ 0.0002, -0.0375,  0.0551],
        [ 0.0002, -0.0375,  0.0551],
        [ 0.0002, -0.0375,  0.0551],
        [ 0.0002, -0.0375,  0.0551],
        [ 0.0002, -0.0375,  0.0551]], device='cuda:0')
tensor(-159.6098, device='cuda:0')
tensor([[ 0.0002, -0.0375,  0.0551],
        [ 0.0002, -0.0375,  0.0551],
        [ 0.0002, -0.0375,  0.0551],
        [ 0.0002, -0.0375,  0.0551],
        [ 0.0002, -0.0375,  0.0551],
        [ 0.0002, -0.0375,  0.0551],
        [ 0.0002, -0.0375,  0.0551],
        [ 0.0002, -0.0375,  0.0551]], device='cuda:0')
tensor(-159.6098, device='cuda:0')
tensor([[ 0.0002, -0.0375,  0.0551],
        [ 0.0002, -0.0375,  0.0551],
        [ 0.0002, -0.0375,  0.0551],
        [ 0.0002, -0.0375,  0.0551],
        [ 0.0002, -0.0375,  0.0551],
        [ 0.0002, -0.0375,  0.0551],
        [ 0.0002, -0.0375,  0.0551],
        

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│ <ipython-input-111-f9b0aff076c7>:1 in <module>                                            │
│ <ipython-input-107-5352bcc6f27d>:23 in testing                                            │
│                                                                                           │
│ /usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py:1051 in _call_impl      │
│                                                                                           │
│   1048 │   │   # this function, and just call forward.                                    │
│   1049 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_h │
│   1050 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):            │
│ ❱ 1051 │   │   │   return forward_call(*input, **kwargs)                                  │
│   1052 │   │   # Do not call functions when jit is used                                   │
│   1053 │   │   full_backward_hooks, non_full_backward_hooks = [], []                      │
│   1054 │   │   if self._backward_hooks or _global_backward_hooks:                         │
│ <ipython-input-100-93fd3087c27c>:51 in forward                                            │
│                                                                                           │
│ /usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py:1051 in _call_impl      │
│                                                                                           │
│   1048 │   │   # this function, and just call forward.                                    │
│   1049 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_h │
│   1050 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):            │
│ ❱ 1051 │   │   │   return forward_call(*input, **kwargs)                                  │
│   1052 │   │   # Do not call functions when jit is used                                   │
│   1053 │   │   full_backward_hooks, non_full_backward_hooks = [], []                      │
│   1054 │   │   if self._backward_hooks or _global_backward_hooks:                         │
│                                                                                           │
│ /home/jupyter/.local/lib/python3.8/site-packages/transformers/models/bert/modeling_bert.p │
│ y:1020 in forward                                                                         │
│                                                                                           │
│   1017 │   │   │   inputs_embeds=inputs_embeds,                                           │
│   1018 │   │   │   past_key_values_length=past_key_values_length,                         │
│   1019 │   │   )                                                                          │
│ ❱ 1020 │   │   encoder_outputs = self.encoder(                                            │
│   1021 │   │   │   embedding_output,                                                      │
│   1022 │   │   │   attention_mask=extended_attention_mask,                                │
│   1023 │   │   │   head_mask=head_mask,                                                   │
│                                                                                           │
│ /usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py:1051 in _call_impl      │
│                                                                                           │
│   1048 │   │   # this function, and just call forward.                                    │
│   1049 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_h │
│   1050 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):            │
│ ❱ 1051 │   │   │   return forward_call(*input, **kwargs)                                  │
│   1052 │   │   # Do not call functions when jit is

In [495]:
#!g1.1
list(t.keys())

['all_probs_sent', 'all_probs_1cat', 'target_sent', 'target_1cat']

In [496]:
#!g1.1
all_probs_sent = t['all_probs_sent']
all_probs_1cat = t['all_probs_1cat']
target_sent = t['target_sent']
target_1cat = t['target_1cat']



In [505]:
#!g1.1


In [503]:
#!g1.1


In [ ]:
#!g1.1


In [ ]:
#!g1.1


In [ ]:
#!g1.1
